# ARIMA, GARCH and Hybrids

Here, in this second notebook, we will discuss two most important and popular models in time series analysis - ARIMA and GARCH. We will implement and discuss how we can forecast univariate and multivariate time series using the same, and discuss their variants and their hybrid use. Further more, we will build upon this discussion in later notebooks where we will use these models in conjunction with other decompositions. For now, we will continue with the touch of the decomposition topic established in the first notebook, for to be used in our Exploratory Data Analysis (EDA).

## Data import and Imputations

We do a repeat of the data we used in notebook 1, the AC Power consumption data in a Hotel.

In [17]:
import datetime
import time
import itertools
import pandas as pd
import numpy as np
import pmdarima as pm
import seaborn as sb
import matplotlib.pyplot as plt
from math import sqrt
from matplotlib.pylab import rcParams

import statsmodels.api as sm
from statsmodels.datasets import macrodata
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.seasonal import seasonal_decompose, STL
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.filters.hp_filter import hpfilter
import statsmodels.tsa.stattools as stools
from sklearn.metrics import mean_squared_error as MSE, r2_score, mean_absolute_percentage_error as MAPE

In [18]:
df = pd.read_csv('data/AC_data.csv')
df = df.rename(columns={'0': 'Datetime'})

df['Datetime']= pd.to_datetime(df.Datetime)
#tdf = df[['Datetime', 'Open', 'Close', 'High', 'Low']]
df

,Datetime,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18
0,2019-08-01 00:00:00,7.518632,8.788315,0.000000,0.000000,2.617045,4.079041,2.782276,4.624447,5.222060,2.151238,1.585072,0.560373,3.142941,2.749470,5.417774,4.113460,3.305072,6.735981
1,2019-08-01 00:01:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2019-08-01 00:02:00,7.426114,8.940615,0.000000,0.000000,2.581625,3.781231,2.529366,5.057423,5.349465,2.414715,2.168184,1.818730,3.085110,2.720484,3.302422,3.986483,3.220588,6.379500
3,2019-08-01 00:03:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2019-08-01 00:04:00,7.052986,9.161103,0.000000,0.000000,2.592095,3.800127,2.332304,6.322521,3.995392,2.237114,3.345624,2.310409,3.132799,2.676861,3.539026,3.797881,3.131560,6.363475
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87835,2019-09-30 23:55:00,0.000000,6.122385,2.192198,2.083315,1.046250,0.000000,3.668421,3.006311,3.614301,1.860847,5.019769,3.154221,3.648026,2.439526,4.062719,1.854352,3.027539,7.259989
87836,2019-09-30 23:56:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
87837,2019-09-30 23:57:00,1.209176,6.152532,2.211421,0.000000,1.448103,0.000000,3.151248,2.871690,3.417942,2.185493,5.174168,2.772349,3.786657,2.060023,4.057525,1.870886,2.991946,7.134647
87838,2019-09-30 23:58:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
df.set_index('Datetime', drop=True, inplace=True)
df = df.resample('2Min').sum()

In [20]:
df.describe()

,AC 1,AC 2,AC 3,AC 4,AC 5,AC 6,AC 7,AC 8,AC 9,AC 10,AC 11,AC 12,AC 13,AC 14,AC 15,AC 16,AC 17,AC 18
count,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000,43920.000000
mean,2.367883,3.850482,2.556306,2.383379,1.415096,2.506499,2.964527,2.536176,3.165096,1.530288,2.392445,2.686915,4.260306,2.902244,4.027397,2.786662,4.199191,6.433688
std,2.277260,2.553006,1.494530,1.800963,0.771625,1.660726,1.486085,1.511464,1.537768,0.692674,1.560585,1.444706,1.865855,1.466128,2.031103,1.614024,1.159760,2.354335
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,2.093627,1.692456,1.238621,0.964796,1.515302,2.176410,1.748428,2.280023,1.110545,1.575432,1.936731,3.243123,1.839066,3.012571,1.865833,3.728744,4.000272
50%,2.081219,3.779912,2.480197,2.300987,1.376681,2.656643,2.996170,2.601814,3.072282,1.500460,2.342875,2.719753,4.208208,2.778750,3.991747,2.665496,4.607949,7.120929
75%,3.653354,5.451999,3.434028,3.470321,1.867081,3.612421,3.890272,3.489748,4.088733,1.960576,3.248267,3.561838,5.373939,3.818686,5.252323,3.688627,4.957075,8.308771
max,11.194789,11.147141,8.430909,9.879500,5.495358,9.929291,9.356776,9.331506,10.026747,4.625030,10.363151,9.887306,11.670892,9.054969,10.349998,10.654430,10.077851,11.979021
